# Problema de selección Quantum Mads (CQM)

---
### Contexto
Un determinado puerto marítimo tiene las siguientes ubicaciones:
- 3 zonas de descarga C1, C2 y C3
- 2 zonas de carga S1, S2
- 2 depósitos para almacenaje D1, D2
- 3 terminales T1, T2 y T3. 

Estas, se nombrarán mediante la siguiente convención sin perdida de generalización:
- C1: **0**; C2: **1**; C3: **2**
- D1: **3**; D2: **4**
- T1: **5**; T2: **6**; T3:**7**
- S1: **8**; S2: **9**

Cada instante de tiempo se reciben y solicitan contenedores vacíos, que se redistribuyen para satisfacer la oferta y la demanda de comercio ultramarino. 

---

### Datos 
Las capacidades máximas de almacenaje tanto en terminales como en depósitos son de 20 contenedores.
Los costes de almacenaje por contenedor son:
- 20 para la terminal T1
- 30 para la terminal T2
- 25 para la terminal T3
- 22 para el depósito D1
- 28 para el depósito D2. 

En un cierto instante de tiempo, cada depósito y cada terminal, excepto la T2, tienen un contenedor vacío en stock. Adicionalmente, en ese mismo instante, cada zona de carga y la terminal T2 reclaman cada una un contenedor vacío para importación ultramarítima, así como cada zona de descarga provee de un contenedor vacío proveniente de comercio de ultra mar. 

Los costes de transporte de contenedores vienen dados por la siguiente matriz agrupada, que contemplan los únicos desplazamientos permitidos (que son a su vez asimétricos): 
- El elemento de la matriz: c[o][d] representa el coste de ir del origen (o) al destino (d)

\begin{bmatrix}
0 & 0 & 0 & 12 & 10 & 14 & 15 & 12 & 0 & 0 \\
0 & 0 & 0 & 19 & 12 & 15 & 17 & 12 & 0 & 0 \\
0 & 0 & 0 & 16 & 18 & 20 & 19 & 17 & 0 & 0 \\
0 & 0 & 0 & 0  & 18 & 18 & 16 & 17 & 11 & 17 \\
0 & 0 & 0 & 16 & 0  & 14 & 17 & 18 & 12 & 14 \\
0 & 0 & 0 & 16 & 15 & 0  & 15 & 18 & 12 & 10 \\
0 & 0 & 0 & 14 & 17 & 15 & 0  & 12 & 19 & 12  \\
0 & 0 & 0 & 13 & 16 & 17 & 16 & 0  & 16 & 18  \\
0 & 0 & 0 & 0  & 0  & 0  & 0  & 0  & 0  & 0  \\
0 & 0 & 0 & 0  & 0  & 0  & 0  & 0  & 0  & 0
\end{bmatrix}

---

### Objetivo

Plantear el problema de optimización para reducir costes y satisfacer la oferta y demanda de contenedores vacíos en ese instante. El resultado ideal debería de ser:
- Movimientos de contenedores que cada solución sugiere
- ¿Es la solución capaz de satisfacer la demanda?
- Coste total asociado.

---

In [1]:
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel, Binary, quicksum
import numpy as np

#Preparación del escenario
origen = [0,1,2,3,4,5,6,7,8,9] #Ubicación inicial
destino = [0,1,2,3,4,5,6,7,8,9] #Ubicación final
costes_de_trayecto = [
    [0, 0, 0, 12, 10, 14, 15, 12, 0, 0],
    [0, 0, 0, 19, 12, 15, 17, 12, 0, 0],
    [0, 0, 0, 16, 18, 20, 19, 17, 0, 0],
    [0, 0, 0, 0, 18, 18, 16, 17, 11, 17],
    [0, 0, 0, 16, 0, 14, 17, 18, 12, 14],
    [0, 0, 0, 16, 15, 0, 15, 18, 12, 10],
    [0, 0, 0, 14, 17, 15, 0, 12, 19, 12],
    [0, 0, 0, 13, 16, 17, 16, 0, 16, 18],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],]
costes_de_almacenamiento = {3: 22, 4: 28, 5: 20, 6: 30, 7: 25}
costes = costes_de_trayecto
for fila in costes: #La matriz final contiene los costes asociados al transporte y almacenamiento final de cada trayecto
    for columna, valor in costes_de_almacenamiento.items():
        fila[columna] += valor  
contenedores = 7 #No estrictamente necesario

#Inicio CQM
cqm=ConstrainedQuadraticModel()

#Una variable BINARIA para cada posible ubicación final de cada ubicación inicial 
#Si x_{o,d} = 0, el trayecto no se selecciona; si x_{o,d} = 1, el trayecto se selecciona
x = [[Binary(f'x_{o},{d}') for d in destino] for o in origen]

#Objetivo: minimizar el coste total de los trayectos seleccionados.
objetivo = quicksum(x[o][d] * costes[o][d] for o in origen for d in destino)
cqm.set_objective(objetivo) 


#Restricciones
#a) Capacidad máxima de almacenamiento: la suma de las variables binarias x[o][d] asociadas al destino d no puede exceder el valor máximo de 20. 
#Esta condición se cumple independientemente de la combinación de trayectos seleccionados.
destinos_almacenaje = [3,4,5,6,7]
for d in destinos_almacenaje:
    cqm.add_constraint(
        quicksum(x[o][d] for o in origen) <= 20, 
        label=f'capacidad-{d}')   
#b) Solicitud de contenedores (demanda): solo una variable x[o][d] asociada a los destinos S1 y S2 puede ser igual a 1. Para el destino T2 al menos una variable debe ser igual a 1.
destinos_demandaS = [8,9]
for d in destinos_demandaS:
    cqm.add_constraint(
        quicksum(x[o][d] for o in origen) == 1, 
        label=f'demanda-carga-{d}')
destinos_demandaT = [6]
for d in destinos_demandaT:
    cqm.add_constraint(
        quicksum(x[o][d] for o in origen) >= 1, 
        label=f'demanda-terminal-{d}')
#c) Provisión de contenedores (oferta): cada origen de descarga que proporciona un contenedor debe asignarse de manera exclusiva a un destino.
origen_descarga = [0,1,2]
for o in origen_descarga:
    cqm.add_discrete([f'x_{o},{d}' for d in destino], 
        label=f'oferta-descarga-{o}')
#d) Condición inicial: garantiza que los almacenes con contenedores inicialmente disponibles se asignan de manera exclusiva a un destino.
#Esta restricción se podría fusionar con la anterior c)
origen_contenedores = [3,4,5,7]
for o in origen_contenedores: 
    cqm.add_discrete([f'x_{o},{d}' for d in destino], 
        label=f'oferta-almacenamiento-{o}')    
#e) Prohibición de permanencia en descargas: los destinos de C1, C2 y C3 no pueden recibir contenedores.
destinos_descarga = [0,1,2]
for d in destinos_descarga:
    cqm.add_constraint(
        quicksum(x[o][d] for o in origen) == 0, 
        label=f'prohibido-descarga-{d}')
#f) Ubicaciones sin contenedores inicialmente: los trayectos con origen en T2, S1 y S2 no están permitidos.
origen_vacio = [6,8,9]
for o in origen_vacio:
    cqm.add_constraint(
        quicksum(x[o][d] for d in destino) == 0, 
        label=f'ubicacion-vacia-{o}')      
#g) Imposibilidad de tránsito Descarga-Carga: los contenedores no se muevan directamente de C a S.
for o in origen_descarga:
    cqm.add_constraint(
        quicksum(x[o][d] for d in destinos_demandaS) == 0, 
        label=f'prohibido-trayecto-{o}') 
#  #h) Número de trayectos debe ser igual al número de contenedores inicialmente en escena. 
#  #Esta restricción es REDUNDANTE por lo que no se incluye.
#  cqm.add_constraint(
#          quicksum(x[o][d] for o in origen for d in destino) == contenedores, 
#          label=f'numero-contenedores')   


# Impresión de todas las restricciones definidas en el modelo
print("Restricciones definidas en el modelo:")
for label, constraint in cqm.constraints.items():
    print(f"{label}: {constraint.to_polystring()}")
      

Restricciones definidas en el modelo:
capacidad-3: x_0,3 + x_1,3 + x_2,3 + x_3,3 + x_4,3 + x_5,3 + x_6,3 + x_7,3 + x_8,3 + x_9,3 <= 20.0
capacidad-4: x_0,4 + x_1,4 + x_2,4 + x_3,4 + x_4,4 + x_5,4 + x_6,4 + x_7,4 + x_8,4 + x_9,4 <= 20.0
capacidad-5: x_0,5 + x_1,5 + x_2,5 + x_3,5 + x_4,5 + x_5,5 + x_6,5 + x_7,5 + x_8,5 + x_9,5 <= 20.0
capacidad-6: x_0,6 + x_1,6 + x_2,6 + x_3,6 + x_4,6 + x_5,6 + x_6,6 + x_7,6 + x_8,6 + x_9,6 <= 20.0
capacidad-7: x_0,7 + x_1,7 + x_2,7 + x_3,7 + x_4,7 + x_5,7 + x_6,7 + x_7,7 + x_8,7 + x_9,7 <= 20.0
demanda-carga-8: x_0,8 + x_1,8 + x_2,8 + x_3,8 + x_4,8 + x_5,8 + x_6,8 + x_7,8 + x_8,8 + x_9,8 == 1.0
demanda-carga-9: x_0,9 + x_1,9 + x_2,9 + x_3,9 + x_4,9 + x_5,9 + x_6,9 + x_7,9 + x_8,9 + x_9,9 == 1.0
demanda-terminal-6: x_0,6 + x_1,6 + x_2,6 + x_3,6 + x_4,6 + x_5,6 + x_6,6 + x_7,6 + x_8,6 + x_9,6 >= 1.0
oferta-descarga-0: x_0,0 + x_0,1 + x_0,2 + x_0,3 + x_0,4 + x_0,5 + x_0,6 + x_0,7 + x_0,8 + x_0,9 == 1.0
oferta-descarga-1: x_1,0 + x_1,1 + x_1,2 + x_1,3 + x_1

In [5]:
#Solución del modelo con LeapHybridCQMSampler
cqm_sampler = LeapHybridCQMSampler(token="DEV-XXX")
sampleset = cqm_sampler.sample_cqm(cqm, label='QuantumMadsCQM')

#Filtro de las soluciones factibles (soluciones que cumplen con todas las restricciones)
soluciones_factibles = sampleset.filter(lambda d: d.is_feasible)

#Verificación de si existen soluciones factibles
if len(soluciones_factibles):

    # Total de soluciones factibles
    numero_soluciones_factibles = len(soluciones_factibles)

    #Solución con menor energía (menor coste), valor de la energía y total de soluciones con dicha energía mínima.
    solucion_energa_minima = soluciones_factibles.first.sample
    energia_minima = min(soluciones_factibles.record['energy'])

    print("Movimientos de contenedores que la solución sugiere:")
    movimientos = []
    coste_total = 0
    demanda_por_destino = {d: 0 for d in destino}  # Para contar la demanda por destino
    for o in origen:
        for d in destino:
            if solucion_energa_minima[f'x_{o},{d}'] == 1:
                movimientos.append((o, d))
                demanda_por_destino[d] += 1
                print(f"- Contenedor de origen {o} a destino {d}, coste: {costes[o][d]}")
                coste_total += costes[o][d]

    #Verificación de si la solución satisface la demanda
    demanda_satisfecha = True
    for d in destino:
        if (d in destinos_demandaS and demanda_por_destino[d] != 1) or ( d in destinos_demandaT and demanda_por_destino[d] < 1):
            demanda_satisfecha = False

    #Resultados finales
    print("\nEvaluación de la solución:")
    print(f"- ¿Satisface la demanda? {'Sí' if demanda_satisfecha == True else 'No'}")
    print(f"- Coste total asociado: {coste_total} unidades")    
else: #len(soluciones_factibles) == 0
    print("No se han encontrado soluciones factibles para el problema.")


#Para mostrar las primeras 5 soluciones del SampleSet (factibles o no)
#print(sampleset.record[:5])

Movimientos de contenedores que la solución sugiere:
- Contenedor de origen 0 a destino 6, coste: 45
- Contenedor de origen 1 a destino 5, coste: 35
- Contenedor de origen 2 a destino 3, coste: 38
- Contenedor de origen 3 a destino 3, coste: 22
- Contenedor de origen 4 a destino 8, coste: 12
- Contenedor de origen 5 a destino 9, coste: 10
- Contenedor de origen 7 a destino 7, coste: 25

Evaluación de la solución:
- ¿Satisface la demanda? Sí
- Coste total asociado: 187 unidades
